<a href="https://colab.research.google.com/github/hoangtheanhhp/ZaloQA/blob/bert_lstm/QASystem/run_zalo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
!rm -rf zalo
!test -d zalo || git clone https://github.com/hoangtheanhhp/ZaloQA zalo
import sys
if 'zalo' not in sys.path:
  sys.path+=['zalo']

Cloning into 'zalo'...
remote: Enumerating objects: 72, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 588 (delta 36), reused 39 (delta 14), pack-reused 516
Receiving objects: 100% (588/588), 94.23 MiB | 23.82 MiB/s, done.
Resolving deltas: 100% (264/264), done.
Checking out files: 100% (37/37), done.


In [26]:
%tensorflow_version 1.x
!pip install tensorflow==1.15.2
!pip install kashgari==1.1.4



In [0]:
DATASET_PATH = "zalo/Dataset/zalo"
SAVE_MODEL_PATH = "/content/drive/My Drive/ZaloAI/result/model"
MODEL_PATH = "/content/drive/My Drive/ZaloAI/bert"

In [35]:
from zalo.QASystem.preprocess import ZaloDatasetProcessor

import kashgari
processor = ZaloDatasetProcessor()
from kashgari.embeddings import BERTEmbedding

bert_embedding = BERTEmbedding(MODEL_PATH,
                               task=kashgari.CLASSIFICATION,
                               sequence_length=128)

tokenizer = bert_embedding.tokenizer

In [36]:



processor.load_from_path(DATASET_PATH, "train")
processor.load_from_path(DATASET_PATH, "val")

data = processor.train_data
val = processor.val_data
train_x = []
train_y = []
val_x = []
val_y = []
for d in data:
  train_x.append(tokenizer.tokenize(d.get('question')) + 
                 tokenizer.tokenize(d.get('text'))[1:])
  train_y.append(str(d.get('label', False)))

for d in val:
  val_x.append(tokenizer.tokenize(d.get('question')) + 
               tokenizer.tokenize(d.get('text'))[1:])
  val_y.append(str(d.get('label', False)))

print(len(train_x), len(train_y), len(val_x), len(val_y))


100%|██████████| 3906/3906 [00:00<00:00, 785376.39it/s]


14200 14200 3906 3906


In [37]:
from tensorflow.python import keras
from kashgari.tasks.classification import BiLSTM_Model
from kashgari.callbacks import EvalCallBack

import logging
logging.basicConfig(level='DEBUG')

model = BiLSTM_Model()

model.fit(train_x,
          train_y,
          val_x,
          val_y,
          batch_size=100)

model.save(SAVE_MODEL_PATH)


Model: "model_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 176)]             0         
_________________________________________________________________
layer_embedding (Embedding)  (None, 176, 100)          720500    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 256)               234496    
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 514       
Total params: 955,510
Trainable params: 955,510
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
143/143 [==============================] - 84s 590ms/step - loss: 0.6022 - acc: 0.6987 - val_loss: 0.6150 - val_acc: 0.6621
Epoch 2/5
143/143 [==============================] - 78s 548ms/step - loss: 0.5435 - acc: 0.7344 - val_loss: 0.5923 

In [38]:
from kashgari.utils import load_model

model = load_model(SAVE_MODEL_PATH)

In [39]:
processor.load_from_path("/content/zalo/Dataset/zalo","test")
test = processor.test_data
test_x = []
test_y = []
for d in test:
  test_x.append(tokenizer.tokenize(d.get('question')) + 
               tokenizer.tokenize(d.get('text'))[1:])
  test_y.append(str(d.get('label')))
# evaluate the model
model.evaluate(test_x, test_y)
# print("loss: %f\n accruracy: %f\nf1_score: %f\nprecision: %f\n, recall: %f\n:" 
#       % (loss, accurany,f1_score,precision, recall))

100%|██████████| 688/688 [00:00<00:00, 164061.69it/s]


              precision    recall  f1-score   support

       False     0.8960    0.8728    0.8842      3128
        True     0.1367    0.1658    0.1498       380

    accuracy                         0.7962      3508
   macro avg     0.5163    0.5193    0.5170      3508
weighted avg     0.8137    0.7962    0.8047      3508

